In [20]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + '/ML dataSets/TV Scripting'
os.chdir(base_dir)

Mounted at /content/gdrive


Now the ML starts, above is the code for colab setup

In [0]:
import keras
import numpy as np
from collections import Counter
from keras.models import Model
from keras.layers import Embedding, Dense, LSTM ,Dropout,Input
from sklearn.model_selection import train_test_split

In [0]:
data_dir=base_dir+"/scripts-utf-8.txt"
with open(data_dir,'r',encoding="utf-8") as f:
    data=f.read()

data=data.lower()

In [41]:
data[:500]

"\n\n\n\n\n\n                    \t\t\t- so, what do you think? - wow.\n it's for you, right? it's for both of us.\n don't go away.\n don't worry.\n there's not enough blood left in my legs to go anywhere.\n hey, it's charlie.\n do your thing when you hear the beep.\n listen, you lousy s.\no.\nb.\n, i will not be treated like this.\n either you call me, or you are gonna be very, very sorry.\n i love you, monkey man.\n charlie? who was that? damn telemarketers.\n a telemarketer who calls you monkey man? i'm on some weir"

In [42]:
data=data[6:] 
data[:100]

"                    \t\t\t- so, what do you think? - wow.\n it's for you, right? it's for both of us.\n d"

In [43]:
data[:400].split("\n")

['                    \t\t\t- so, what do you think? - wow.',
 " it's for you, right? it's for both of us.",
 " don't go away.",
 " don't worry.",
 " there's not enough blood left in my legs to go anywhere.",
 " hey, it's charlie.",
 ' do your thing when you hear the beep.',
 ' listen, you lousy s.',
 'o.',
 'b.',
 ', i will not be treated like this.',
 ' either you call me, or you are gonna be very, very sorry.',
 ' i love you, monkey man.',
 ' charlie']

We are getting S.O.B as different

In [0]:
punch=['.','[',']','(',')',';',':',"'","/",'"',',','?','*','!','-','$','%',
       '&','\n']

for i in punch:
    data=data.replace(i,' '+i+' ')
data=data.replace('\n','<newline>')

In [45]:
data[:400]

"                    \t\t\t -  so ,  what do you think ?   -  wow .  <newline>  it ' s for you ,  right ?  it ' s for both of us .  <newline>  don ' t go away .  <newline>  don ' t worry .  <newline>  there ' s not enough blood left in my legs to go anywhere .  <newline>  hey ,  it ' s charlie .  <newline>  do your thing when you hear the beep .  <newline>  listen ,  you lousy s .  <newline> o .  <new"

In [0]:
def get_vocab(text):
    counter=Counter()
    int_to_vocab=dict()
    vocab_to_int=dict()
    
    for d in text.split():
        counter[d]+=1
    
    index=0
    for word in counter:
        int_to_vocab[index]=word
        vocab_to_int[word]=index
        index+=1
    
    return counter, vocab_to_int, int_to_vocab

In [0]:
vocab,vocab_to_int, int_to_vocab =get_vocab(data)

In [48]:
vocab

Counter({'-': 8114,
         'so': 1065,
         ',': 12847,
         'what': 2158,
         'do': 1213,
         'you': 8596,
         'think': 549,
         '?': 6814,
         'wow': 47,
         '.': 23047,
         '<newline>': 9250,
         'it': 3225,
         "'": 11528,
         's': 4210,
         'for': 1291,
         'right': 721,
         'both': 64,
         'of': 1825,
         'us': 196,
         'don': 1353,
         't': 2811,
         'go': 777,
         'away': 90,
         'worry': 70,
         'there': 611,
         'not': 1290,
         'enough': 75,
         'blood': 22,
         'left': 71,
         'in': 1560,
         'my': 1531,
         'legs': 12,
         'to': 4079,
         'anywhere': 16,
         'hey': 570,
         'charlie': 1059,
         'your': 1494,
         'thing': 219,
         'when': 416,
         'hear': 80,
         'the': 4267,
         'beep': 6,
         'listen': 133,
         'lousy': 7,
         'o': 17,
         'b': 8,
        

In [49]:
print("vocab size : ",len(vocab))

vocab size :  9000


Coverting the whole text corpus into integer corpus

In [0]:
text_int=[]

for word in data.split():
    text_int.append(vocab_to_int[word])
text_int=np.array(text_int)

In [51]:
len(text_int)

259916

In [52]:
text_int[:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 0, 8])

In [53]:
int_to_vocab

{0: '-',
 1: 'so',
 2: ',',
 3: 'what',
 4: 'do',
 5: 'you',
 6: 'think',
 7: '?',
 8: 'wow',
 9: '.',
 10: '<newline>',
 11: 'it',
 12: "'",
 13: 's',
 14: 'for',
 15: 'right',
 16: 'both',
 17: 'of',
 18: 'us',
 19: 'don',
 20: 't',
 21: 'go',
 22: 'away',
 23: 'worry',
 24: 'there',
 25: 'not',
 26: 'enough',
 27: 'blood',
 28: 'left',
 29: 'in',
 30: 'my',
 31: 'legs',
 32: 'to',
 33: 'anywhere',
 34: 'hey',
 35: 'charlie',
 36: 'your',
 37: 'thing',
 38: 'when',
 39: 'hear',
 40: 'the',
 41: 'beep',
 42: 'listen',
 43: 'lousy',
 44: 'o',
 45: 'b',
 46: 'i',
 47: 'will',
 48: 'be',
 49: 'treated',
 50: 'like',
 51: 'this',
 52: 'either',
 53: 'call',
 54: 'me',
 55: 'or',
 56: 'are',
 57: 'gonna',
 58: 'very',
 59: 'sorry',
 60: 'love',
 61: 'monkey',
 62: 'man',
 63: 'who',
 64: 'was',
 65: 'that',
 66: 'damn',
 67: 'telemarketers',
 68: 'a',
 69: 'telemarketer',
 70: 'calls',
 71: 'm',
 72: 'on',
 73: 'some',
 74: 'weird',
 75: 'list',
 76: 'okay',
 77: 'woman',
 78: 'went',
 79:

These are approximately number of words in Two and Half Men's 3 Sesaons

Tune these below hyper parameters

In [0]:
vocab_size=len(vocab)
seq_len=5
embedding=300
lstm_size=128


### Notice that in the tutorial it is just one step ahead and ahead, I am making seq_len steps ahead, and see what that happens

In [0]:
def get_training_data(data,seq_len):
    x_train=[]
    y_train=[]
    
    for i in range(0,len(data)-seq_len):
        x=data[i:i+seq_len]
        y=data[i+1:i+seq_len+1]
        
        x_train.append(np.array(x))
        y_train.append(np.array(y))
    return x_train, y_train

In [0]:
x,y =get_training_data(text_int,seq_len)

x=np.array(x)
y=np.array(y)
y=y.reshape(y.shape[0],y.shape[1],1)

In [81]:
x.shape, y.shape

((259911, 5), (259911, 5, 1))

In [0]:
inp=Input((None,))

embed=Embedding(input_dim=vocab_size,output_dim=embedding)
lstm1=LSTM(lstm_size,return_sequences=True,return_state=True)
lstm2=LSTM(lstm_size,return_sequences=True,return_state=True)
lstm3=LSTM(lstm_size,return_sequences=True,return_state=True)
dense=Dense(vocab_size)

net=embed(inp)
net, h1, c1=lstm1(net)
net, h2, c2=lstm2(net)
net, h3, c3=lstm3(net)
out=dense(net)

model=Model(inp,out)

init_states=[Input((lstm_size,)) for i in range(6)]

inference=embed(inp)
inference, h1, c1=lstm1(inference,initial_state=init_states[:2])
inference, h2, c2=lstm2(inference,initial_state=init_states[2:4])
inference, h3, c3=lstm3(inference,initial_state=init_states[4:6])
inf_out=dense(inference)

states=[h1,c1,h2,c2,h3,c3]
inf_model=Model([inp]+ init_states,[inf_out]+ states)

In [0]:
from keras import optimizers
rmsprop = optimizers.RMSprop(lr=0.01)

In [0]:
# model.optimzer.lr=.01
model.compile(optimizer=rmsprop,loss='sparse_categorical_crossentropy'
              ,metrics=['accuracy'])

**Make validation set and change the hyper params as per best accuracy and see the things**

In [85]:
print(model.fit(x,y,batch_size=128,epochs=1,shuffle=True))
# model.save(model.model)

Epoch 1/1
259911/259911 [==============================] - 93s 360us/step - loss: 9.1148 - acc: 0.0032


When the model is finalised then extract the text

In [0]:
# def extract_text(length,start):
    
#     states=[np.zeros((1,lstm_size)) for i in range(6)]
    
#     token=np.zeros((1,1))
#     token[0,0]=start
#     text=int_to_vocab[start] + ' '
    
#     for i in range(length):
        
#         out=inf_model.predict([token]+states)
#         word=np.argmax(out[0][0,0,:])
#         text+=" "+ int_to_vocab[word]
#         states=out[1:7]
#         token[0][0]=word
#     return text

In [0]:
def extract_text(length, states):
    
#     states = []
#     for a in len(start.strip(" ")):
#       states.append("a")
  
#     for i in range(len(start.strip(" "))):
#       states[]
    states="some String"
    states=["hello","charlie","how","are","are","you"] 
    int_states=[]
    for s in states:
      int_states.append(vocab_to_int[s])
    int_states.pop()
    print(int_states)
    token = np.zeros((1,1))
#     token[0,0] = start
    text = ' '.join(states)
  
    token[0,0]=int_states[0]
    int_states=np.array(int_states)
    for i in range(length):
        
        out = inf_model.predict([token]+int_states)
        word = np.argmax(out[0][0,0,:])
        text += int_to_vocab[word] + ' '
        states = out[1:7]
        token[0][0] = word
        
    return text    

In [102]:
generate_text=extract_text(100,0) # we are getting this error because we have to send the embedded words rather than just the 

[416, 35, 151, 56, 56]


ValueError: ignored

In [0]:
def post_process_text(text):
    
    punch1=['.', ':', '!', ';', ')', ']', '?', ',', '%']
    for i in punch1:
        text = text.replace(' '+i, i)
    punch2 = ['[', '(', '$']    
    for i in punch2:
        text = text.replace(i+' ', i)
    punch3 = ["'", '-']    
    for i in punch3:
        text = text.replace(' '+i+' ', i)
        
    cleanText=""
    text = text.split('<newline>')  
    for line in text:
        if len(line):
          cleanText+=line+"\n"
    return cleanText

In [90]:
print(post_process_text(generate_text))

- upkeep upkeep upkeep upkeep upkeep upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep sided upkeep 

